In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import urllib.request
import json
import os
import ssl
import pandas as pd
import numpy as np

In [ ]:
def allowSelfSignedHttps(allowed):
    # bypass the server certificate verification on client side
    if allowed and not os.environ.get('PYTHONHTTPSVERIFY', '') and getattr(ssl, '_create_unverified_context', None):
        ssl._create_default_https_context = ssl._create_unverified_context
deployment1 = 'http://83f76041-cac5-4f06-b7e0-22eb771afb13.southeastasia.azurecontainer.io/score'
deployment2 = 'http://899cab75-618f-4543-adef-85d8edc9bff1.southeastasia.azurecontainer.io/score'
deployment3 = 'http://45f246cd-5d1a-49aa-a5e1-cf17c4539fff.southeastasia.azurecontainer.io/score' #mol2vec
deployment4 = 'http://c709b7ea-9032-494d-bd4f-c4c4cd06f609.southeastasia.azurecontainer.io/score' #gcnconcat
deployment5 = 'http://d0d7d9d4-aac6-4755-90c0-24a5ee50d82a.southeastasia.azurecontainer.io/score' #sageconcat
deployment6 = 'http://324edf84-c2cc-4298-b9f3-eba05dbe657b.southeastasia.azurecontainer.io/score' #sage+gcn concat linker
deployment7 = 'http://c4e43588-546f-4737-9a4d-6e8171838752.southeastasia.azurecontainer.io/score' #grid lasthope
#deployment8 = 'http://1cf2cb39-65ff-4bd7-9b41-41e043bbe38c.southeastasia.azurecontainer.io/score' #sayonara
def send(datajson):
    allowSelfSignedHttps(True) # this line is needed if you use self-signed certificate in your scoring service.

    # Request data goes here
    data = datajson

    body = str.encode(json.dumps(data))

    url = deployment7
    api_key = '' # Replace this with the API key for the web service
    headers = {'Content-Type':'application/json', 'Authorization':('Bearer '+ api_key)}

    req = urllib.request.Request(url, body, headers)

    try:
        response = urllib.request.urlopen(req)
        print(response)
        result = response.read()
        res2 = json.loads(result)
        res = json.loads(res2)
        return res['result']
    except urllib.error.HTTPError as error:
        print("The request failed with status code: " + str(error.code))

        # Print the headers - they include the requert ID and the timestamp, which are useful for debugging the failure
        print(error.info())
        print(json.loads(error.read().decode("utf8", 'ignore')))

In [ ]:
file_path='/content/drive/MyDrive/tmlcc-2021/preprocess/auto_feat_last_hope_test.csv' #change your file path to auto_feat_last_hope_test.scv
data = pd.read_csv(file_path)
print(data.columns)

In [ ]:
ans = pd.DataFrame()
ans['id'] = [str(i) for i in range(68614,85614)]

In [ ]:
mof = data['mofname'].tolist()
data.drop('mofname', axis=1, inplace=True)

In [ ]:
datajson = {"data": data.iloc[0:].to_dict(orient='records')}

In [ ]:
lis = send(datajson)
len(lis)

In [ ]:
oldsub = pd.read_csv('/content/drive/MyDrive/tmlcc-2021/preprocess/xgboost_submission_hist.csv') #change your file path to xgboost_submission_hist.csv
oldsub.head()

In [ ]:
inlist = [int(i[9:]) for i in mof]
i=68614
j=0
anslis = []
while(True):
    if(i==85614):
      break
    if(i not in inlist):
      anslis.append(float(oldsub.iloc[i-68614]['CO2_working_capacity [mL/g]']))
    else:
      anslis.append(lis[j])
      j+=1
    i+=1
print(len(anslis))

In [ ]:
ans['CO2_working_capacity [mL/g]'] = anslis

In [ ]:
ans.to_csv (r'/content/drive/MyDrive/tmlcc-2021/testset/finalhuh.csv', index = False, header=True)